In [2]:
from huggingface_hub import hf_hub_download

ltxv_model_path = hf_hub_download(
    repo_id="Lightricks/LTX-Video",
    filename="./ltxv-2b-0.9.8-distilled.safetensors",
    repo_type="model",
)

ltxv-2b-0.9.8-distilled.safetensors:   0%|          | 0.00/6.34G [00:00<?, ?B/s]

In [1]:
from ltx_video.models.autoencoders.causal_video_autoencoder import CausalVideoAutoencoder

device = 'cuda'

ltxv_model_path = './ltxv-2b-0.9.8-distilled.safetensors'
vae = CausalVideoAutoencoder.from_pretrained(ltxv_model_path)
vae.to(device)
print("-")

-


In [2]:
from torchvision.io import read_video
from einops import rearrange
import torch

vid_path = '/workspace/sixteen128.mp4'

video_frames, audio, info = read_video(vid_path)
print("info : ", info)

print(video_frames.shape, video_frames.dtype)  # (T, H, W, C)  # T: frame count, C=3
video_frames = rearrange(video_frames.unsqueeze(dim=0).tile(2, 1, 1, 1, 1), 'b t h w c -> b c t h w').to(device).to(torch.float32)
video_frames = video_frames[:, :, :65, :, :] # 8의 배수 + 1이어야 한다.
print(video_frames.shape, video_frames[0][0][0][0])

video_frames = (video_frames/255) * 2 - 1

info :  {'video_fps': 16.0}
torch.Size([80, 128, 128, 3]) torch.uint8
torch.Size([2, 3, 65, 128, 128]) tensor([224., 235., 222., 190., 146., 122.,  94.,  72.,  47.,  46.,  44.,  42.,
         37.,  35.,  31.,  31.,  31.,  33.,  33.,  33.,  30.,  29.,  29.,  30.,
         30.,  29.,  29.,  29.,  29.,  29.,  27.,  26.,  24.,  24.,  24.,  24.,
         26.,  26.,  26.,  26.,  24.,  24.,  24.,  22.,  24.,  25.,  29.,  32.,
         49., 113., 228., 221., 224., 218., 227., 230., 230., 230., 227., 227.,
        227., 226., 226., 226., 226., 226., 226., 226., 226., 226., 225., 225.,
        224., 224., 228., 228., 129.,  51.,  28.,  53.,  76.,  69.,  72.,  67.,
         69.,  69.,  69.,  69.,  69.,  69.,  69.,  69.,  69.,  69.,  69.,  68.,
         69.,  69.,  68.,  68.,  59.,  31.,  15.,  19.,  18.,  18.,  18.,  18.,
         18.,  18.,  18.,  18.,  20.,  20.,  20.,  20.,  20.,  20.,  20.,  20.,
         20.,  20.,  20.,  20.,  20.,  20.,  20.,  20.], device='cuda:0')


/usr/local/lib/python3.11/dist-packages/torchvision/io/video.py:197: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


In [3]:
with torch.no_grad():
    aeoutput = vae.encode(video_frames)
    latent = aeoutput.latent_dist.mode()
    print(latent.shape)

del aeoutput
torch.cuda.empty_cache()

torch.Size([2, 128, 9, 4, 4])


In [4]:
timestep = torch.ones(video_frames[:, :, :49, :, :].shape[0], device=device) * 0.1

reconstructed_videos = vae.decode(
    latent[:, :, :4, :, :], target_shape=video_frames[:, :, :49, :, :].shape, timestep=timestep
).sample

print(reconstructed_videos.shape)

torch.Size([2, 3, 25, 128, 128])


In [5]:
import torchvision
from torchvision.io import read_video, write_video

output_path = './reencoded_video12816test_2.mp4'
recon_videos = (rearrange(reconstructed_videos, "b c t h w -> b t h w c")[0].cpu()/2 + 0.5) * 255

write_video(
    filename=output_path,
    video_array=recon_videos,      # shape: (T, H, W, C)
    fps=int(info['video_fps']),
    video_codec='libx264',         # optional
    options={"crf": "18"}          # optional: quality setting
)

In [9]:
recon_videos[0][0][0]

tensor([255, 255, 255], dtype=torch.uint8)

In [11]:
video_frames[0][0][0][0]

tensor([217., 230., 239., 238., 237., 225., 217., 201., 166., 155., 144., 133.,
        114., 100.,  81.,  67.,  56.,  51.,  45.,  41.,  40.,  39.,  38.,  36.,
         37.,  37.,  36.,  33.,  28.,  27.,  30.,  33.,  33.,  33.,  33.,  33.,
         33.,  34.,  33.,  33.,  33.,  31.,  30.,  30.,  30.,  30.,  29.,  29.,
         29.,  29.,  29.,  29.,  29.,  29.,  29.,  29.,  28.,  28.,  28.,  28.,
         27.,  27.,  27.,  27.,  27.,  22.,  17.,  20.,  26.,  27.,  27.,  27.,
         27.,  27.,  27.,  27.,  27.,  27.,  27.,  27.,  26.,  26.,  26.,  26.,
         26.,  26.,  26.,  26.,  24.,  26.,  26.,  27.,  29.,  30.,  33.,  35.,
         38.,  41.,  83., 163., 227., 227., 223., 223., 227., 222., 206., 214.,
        227., 227., 228., 228., 226., 227., 227., 227., 227., 227., 227., 227.,
        227., 227., 227., 227., 227., 227., 227., 227., 226., 226., 226., 226.,
        226., 226., 226., 226., 226., 226., 226., 226., 226., 226., 225., 225.,
        224., 224., 222., 221., 220., 22

In [13]:
import torch.nn as nn

# spatial 4×4를 1×1로 줄이기 위한 3D‑Conv
#   kernel_size=(1,4,4), stride=(1,4,4)
conv3d = nn.Conv3d(
    in_channels=128,
    out_channels=128,
    kernel_size=(1, 4, 4),
    stride=(1, 4, 4),
    padding=0,
).to(device)

y = conv3d(latent).squeeze()
print(y.shape)

torch.Size([2, 128, 9])


In [19]:
import torch.nn as nn

class GRN(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.gamma = nn.Parameter(torch.zeros(1, 1, dim))
        self.beta = nn.Parameter(torch.zeros(1, 1, dim))

    def forward(self, x):
        Gx = torch.norm(x, p=2, dim=1, keepdim=True)
        Nx = Gx / (Gx.mean(dim=-1, keepdim=True) + 1e-6)
        return self.gamma * (x * Nx) + self.beta + x

class ConvNeXtV2Block(nn.Module):
    def __init__(
        self,
        dim: int,
        intermediate_dim: int,
        dilation: int = 1,
    ):
        super().__init__()
        padding = (dilation * (7 - 1)) // 2
        self.dwconv = nn.Conv1d(
            dim, dim, kernel_size=7, padding=padding, groups=dim, dilation=dilation
        )  # depthwise conv
        self.norm = nn.LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, intermediate_dim)  # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.grn = GRN(intermediate_dim)
        self.pwconv2 = nn.Linear(intermediate_dim, dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        input x is sequence, tensor of (bs, seq_len, dim)
        """
        residual = x
        x = x.transpose(1, 2)  # b n d -> b d n
        x = self.dwconv(x)
        x = x.transpose(1, 2)  # b d n -> b n d
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.grn(x)
        x = self.pwconv2(x)
        return residual + x

cnv = ConvNeXtV2Block(128, 64).to(device)

In [20]:
# x = torch.randn((2, 13, 32))
out = cnv(rearrange(y, 'b d n -> b n d'))
print(out.shape)

torch.Size([2, 9, 128])


In [25]:
import torch.nn as nn

class UpsampleHalveChannel(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.ConvTranspose1d(
            in_channels=in_ch,
            out_channels=out_ch,
            kernel_size=4,
            stride=2,
            padding=1,
        )
        self.norm = nn.LayerNorm(out_ch)
        self.act = nn.GELU()

    def forward(self, x):
        x = x.transpose(1, 2)
        x = self.up(x)                   # → [B, C/2, 2L]
        x = x.transpose(1, 2)            # → [B, 2L, C/2]
        x = self.norm(x)
        return self.act(x)

upconv = UpsampleHalveChannel(128, 64).to(device)

In [26]:
upconv(rearrange(y, 'b d n -> b n d')).shape

torch.Size([2, 18, 64])

In [29]:
layers = nn.Linear(128, 64)

In [31]:
total_params = sum(p.numel() for p in conv3d.parameters())
print(f"conv3d parameters: {total_params:,}")

total_params = sum(p.numel() for p in cnv.parameters())
print(f"cnv parameters: {total_params*4:,}")

total_params = sum(p.numel() for p in layers.parameters())
print(f"layers parameters: {total_params:,}")

conv3d parameters: 262,272
cnv parameters: 71,936
layers parameters: 8,256
